# FIFA 23 - Ultimate team players

Accessing the Futbin website to collect data of Players in FIFA 23 Ultimate Team.\

### Libs
- Pandas - To load and manipulate data
- Scrapy - To scrap data from a website

In [17]:
import pandas as pd
from selenium import webdriver    
from datetime import  date
from scrapy import Selector
import time
import warnings
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
import random

# defining today to use in some cases
today = date.today()

df_players = pd.DataFrame()

# Step 1 - Iterate over pages to get players data and sabe it on dataframe
We need to instanciate a new driver every iteration to by-pass some anti-scrap blockers.

In [ ]:
# variable to pagination
page_number = 1

for page in range(185):
    # Creating a webdriver instance
    driver = webdriver.Chrome(r"Users\Felipe Mayer\Downloads\chromedriver_win32")
    url = 'https://www.futbin.com/players?page=' + str(page_number)
            
    driver.get(url)
    time.sleep(random.randint(2, 5))

    src = driver.page_source
    sel = Selector(text=src)

    players = sel.xpath('.//thead/following-sibling::tbody/tr[starts-with(@class, "player_tr")]')

    for player in players:
        index = len(df_players.index)
        df_players.loc[index, 'Name'] = player.xpath('.//a[contains(@class, "player_name_players_table")]/text()').get(default='')
        df_players.loc[index, 'Team'] = player.xpath('.//span[@class="players_club_nation"]/a[1]/@data-original-title').get(default='')
        df_players.loc[index, 'Country'] = player.xpath('.//span[@class="players_club_nation"]/a[2]/@data-original-title').get(default='')
        df_players.loc[index, 'League'] = player.xpath('.//span[@class="players_club_nation"]/a[3]/@data-original-title').get(default='')
        df_players.loc[index, 'Rating'] = player.xpath('.//span[contains(@class, "rating")]/text()').get(default='')
        rating_type = player.xpath('.//span[contains(@class, "rating")]/@class').get(default='')
        s = 'ut23'
        ss = rating_type.split(s, 1)
        df_players.loc[index, 'Rare'] = rating_type.split(s, 1)[1].split()[-1]
        df_players.loc[index, 'Version color'] = ss[1].split(df_players.loc[index, 'Rare'], 1)[0]
        df_players.loc[index, 'Position'] = player.xpath('./td[4]/div[1]/text()').get(default='')
        df_players.loc[index, 'Other Positions'] = player.xpath('./td[4]/div[2]/text()').get(default='')
        df_players.loc[index, 'Version'] = player.xpath('./td[5]/div[1]/text()').get(default='')
        df_players.loc[index, 'Run type'] = player.xpath('./td[5]/div[2]/a/text()').get(default='').replace('/n', ' ').strip()
        df_players.loc[index, 'Price'] = player.xpath('./td[6]/span/text()').get(default='')
        df_players.loc[index, 'Skill'] = player.xpath('./td[7]/text()').get(default='')
        df_players.loc[index, 'Weak foot'] = player.xpath('./td[8]/text()').get(default='')
        df_players.loc[index, 'Attack rate'] = player.xpath('./td[9]/span[1]/text()').get(default='')
        df_players.loc[index, 'Defense rate'] = player.xpath('./td[9]/span[2]/text()').get(default='')
        df_players.loc[index, 'Pace'] = player.xpath('./td[10]/span/text()').get(default='')
        df_players.loc[index, 'Shoot'] = player.xpath('./td[11]/span/text()').get(default='')
        df_players.loc[index, 'Pass'] = player.xpath('./td[12]/span/text()').get(default='')
        df_players.loc[index, 'Drible'] = player.xpath('./td[13]/span/text()').get(default='')
        df_players.loc[index, 'Defense'] = player.xpath('./td[14]/span/text()').get(default='')
        df_players.loc[index, 'Physical'] = player.xpath('./td[15]/span/text()').get(default='')
        df_players.loc[index, 'Body type'] = player.xpath('./td[16]/div[2]/a/text()').get(default='')
        df_players.loc[index, 'Height (cm)'] = player.xpath('./td[16]/div[1]/text()').get(default='').split('|')[0].replace('cm', ' ').strip()
        weight = player.xpath('string(./td[16]/div[2])').get(default='')
        if weight != '':
            df_players.loc[index, 'Weight (kg)'] = weight.split('(')[1].replace('kg)', ' ').strip()
        df_players.loc[index, 'Popularity'] = player.xpath('./td[17]/text()').get(default='')
        df_players.loc[index, 'Base Stats'] = player.xpath('./td[18]/text()').get(default='')
        df_players.loc[index, 'In Game Stats'] = player.xpath('./td[19]/text()').get(default='')
        df_players.loc[index, 'Game version'] = 'FIFA 23'
        df_players.loc[index, 'Updated at'] = today

    driver.close()

    page_number += 1

display(df_players)

# Step 2 - Save it on a CSV to be used in futher analysis

In [20]:
df_players.to_csv('players.csv')